<a href="https://colab.research.google.com/github/AmulyaThammineni/6440304_Thammineni_Naga_Amulya/blob/main/lipspeechc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get -qq install ffmpeg

In [3]:
!pip uninstall -y numpy protobuf


Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3


In [1]:
!pip install numpy==1.24.4

In [5]:
!pip install protobuf==4.25.3

^C


In [2]:
!pip install opencv-python-headless
!pip install gradio

In [3]:
!pip install torchvision

In [4]:
!pip install matplotlib


In [1]:
!pip install mediapipe==0.10.9


In [2]:
!pip install transformers

In [3]:
!pip install torchaudio

In [4]:
import os
import cv2
import torch
import numpy as np
from PIL import Image

In [5]:
import gradio as gr
import mediapipe as mp

In [6]:
from torchvision import transforms

In [7]:
from transformers import Wav2Vec2Processor, HubertForCTC


In [8]:
processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


HubertForCTC(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertLayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in

In [9]:
transform = transforms.Compose([
    transforms.ToTensor()
])

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False)


In [10]:
def extract_lip_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)
        if results.multi_face_landmarks:
            h, w, _ = frame.shape
            x_min, y_min, x_max, y_max = w, h, 0, 0
            for lm in results.multi_face_landmarks[0].landmark[61:89]:
                x, y = int(lm.x * w), int(lm.y * h)
                x_min, y_min = min(x_min, x), min(y_min, y)
                x_max, y_max = max(x_max, x), max(y_max, y)
            crop = frame[y_min:y_max, x_min:x_max]
            if crop.size:
                crop = cv2.resize(crop, (100, 50))
                frames.append(crop)
    cap.release()
    return frames

In [16]:
def predict_text_from_video(video_file):
    import tempfile
    import torchaudio
    import shutil

    # Create a temp copy of the uploaded video
    tmp_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name
    shutil.copy(video_file, tmp_path)  # ✅ because video_file is already a path string

    # Extract lip frames (visual cue, optional)
    frames = extract_lip_frames(tmp_path)
    if not frames:
        return "No lip movement detected."

    # Extract audio
    audio_path = tmp_path.replace(".mp4", ".wav")
    os.system(f"ffmpeg -y -i {tmp_path} -ar 16000 -ac 1 -f wav {audio_path}")
    waveform, sample_rate = torchaudio.load(audio_path)

    # Transcribe using AV-HuBERT
    inputs = processor(waveform.squeeze(), sampling_rate=sample_rate, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription.strip()


In [ ]:
iface = gr.Interface(
    fn=predict_text_from_video,
    inputs=gr.Video(label="Upload a video file (.mp4)"),  # ✅ No `type` parameter
    outputs=gr.Textbox(label="Predicted Text Output"),
    title="Real-Time Lip-to-Text Transcriber (AV-HuBERT)",
    description="This demo uses AV-HuBERT to convert spoken audio from a video into text. Lip movements are tracked for reference only. For full visual lip-reading, use a vision-only AV-HuBERT variant."
)

iface.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f1c3f2685f0b788f09.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
